# Loading some packages

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.feature_selection import r_regression, SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

from scipy import stats

from statsmodels.stats.outliers_influence import variance_inflation_factor

from boruta import BorutaPy

from BorutaShap import BorutaShap

from collections import Counter

import os
from pathlib import Path

import warnings 
warnings.filterwarnings('ignore')

c:\Users\vchar\anaconda3\envs\ml_projects\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [81]:
import boruta

boruta.__file__

'c:\\Users\\vchar\\anaconda3\\envs\\ml_projects\\lib\\site-packages\\boruta\\__init__.py'

# Loading the data

In [9]:
DATA_PATH = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\portfolio/\ArmenianHousePrices/\notebooks/\data')

data = pd.read_csv(os.path.join(DATA_PATH, 'final_data.csv'))

cat_feats_list = list(data.select_dtypes(include=['object']).columns)
remove_cat_list =[
    'description', 'address', 'seller_type', 'total_description',
    'renewed_date', 'seller_id', 'region', 'geo_location', 'estate_type'
]
cat_feats_list = [i for i in cat_feats_list if i not in remove_cat_list]
cat_feats_list = ['Number of Rooms',
 'Renovation',
 'Furniture',
 'New Construction',
 'Construction Type',
 'Number of Bathrooms',
 'Balcony',
 'Elevator']
print(cat_feats_list)

ordinal_feats_list = ['Number of Rooms', 'Number of Bathrooms']

nominal_feats_list = [i for i in cat_feats_list if i not in ordinal_feats_list]

num_feats_list = list(data.select_dtypes(exclude=['object']).columns)
remove_num_feats_list = ['posted_date', 'sqm_price_usd', 'estate_id', 'latitude', 'longitude', 'geo_location']
num_feats_list = [i for i in num_feats_list if i not in remove_num_feats_list and i not in cat_feats_list]
# feats_name_list = [col for col in data.columns if col!='']
print(num_feats_list)

feats_name_list = []
feats_name_list.extend(cat_feats_list)
feats_name_list.extend(num_feats_list)

y = data['sqm_price_usd'].copy().to_frame()
X = data[feats_name_list].copy()

['Number of Rooms', 'Renovation', 'Furniture', 'New Construction', 'Construction Type', 'Number of Bathrooms', 'Balcony', 'Elevator']
['Ceiling Height', 'Floor', 'Floor Area', 'Floors in the Building', 'covered parking', 'outdoor parking', 'garage', 'air conditioner', 'dishwasher', 'washing machine', 'fridge', 'drying machine', 'stove', 'park view', 'street view', 'view of Ararat', 'yard view', 'city view', 'concierge', 'intercom entry', 'playground', 'is_capital', 'city_attr_0', 'city_attr_1', 'city_attr_2', 'city_attr_3', 'city_attr_4', 'city_attr_5', 'city_attr_6', 'city_attr_7', 'city_attr_8', 'city_attr_9', 'city_attr_10', 'city_attr_11', 'metro_station_0', 'metro_station_1', 'metro_station_2', 'metro_station_3', 'metro_station_4', 'metro_station_5', 'metro_station_6', 'metro_station_7', 'metro_station_8', 'metro_station_9', 'closest_metro_distance', 'mall_0', 'mall_1', 'mall_2', 'mall_3', 'closest_mall_distance']


In [10]:
X.head()

,Number of Rooms,Renovation,Furniture,New Construction,Construction Type,Number of Bathrooms,Balcony,Elevator,Ceiling Height,Floor,...,metro_station_6,metro_station_7,metro_station_8,metro_station_9,closest_metro_distance,mall_0,mall_1,mall_2,mall_3,closest_mall_distance
0,2,0,0,0,0,1,0,0,2.75,3,...,61.169827,53.006584,29.637074,244.102697,24.408433,186.156946,129.294787,124.079898,293.619775,124.079898
1,3,1,0,0,0,1,1,0,2.75,14,...,139.532072,103.661085,49.488490,196.393410,22.690618,134.940334,208.131945,157.861426,300.968318,134.940334
2,2,2,1,0,1,1,1,1,2.80,4,...,273.024165,324.852997,275.092613,183.363983,145.966273,198.004532,311.409442,154.460103,565.084056,154.460103
3,3,3,2,0,0,1,2,1,2.60,1,...,334.600766,265.608187,313.773654,514.741262,265.608187,454.336069,344.228264,440.023840,23.111663,23.111663
4,3,4,0,0,0,1,0,0,2.80,4,...,131.678114,83.056489,42.581168,222.421567,42.581168,160.966858,197.407935,170.063984,275.782140,160.966858


# Feature elimination

## Removing low variance features

In [13]:
variance_selector = VarianceThreshold(threshold=0.05)
variance_selector.fit(X[num_feats_list])
# X_selection = variance_selector.fit_transform(X[num_feats_list])
var_selected_feats = X[num_feats_list].iloc[:, list(variance_selector.get_support())].columns.to_list()
print('Numeric features to remove:')
[col for col in num_feats_list if col not in var_selected_feats]

Numeric features to remove:


['Ceiling Height', 'covered parking', 'garage', 'drying machine']

In [14]:
len(num_feats_list)

61

In [13]:
X_selection.shape

(22157, 56)

## Variance Inflation Factor

In [ ]:
def calculate_vif(X):
    
    vif = pd.DataFrame()
    vif["features"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return vif

def select_feats_via_vif(X):

    vif_df = calculate_vif(X)

    while vif_df[vif_df['VIF'] >=10].shape[0] != 0:
        vif_df.sort_values('VIF', ascending=False).reset_index(drop=True, inplace=True)
        elimination_candidate = vif_df.iloc[0]['features']
        new_X = X.drop(columns=elimination_candidate)
        vif_df = calculate_vif(new_X)

    return list(vif_df['features'].values)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=3, direction=”forward”)
sfs.fit(X, y)
X_selection = sfs.transform(X)

svc = SVC(kernel="linear")
rfe = RFE(svc, n_features_to_select=3)
rfe.fit(X, y)
X_selection = rfe.transform(X)

## Feature selection class

In [21]:
selector = SelectKBest(r_regression, k=15)
selector.fit_transform(X[num_feats_list], y)
selected_feats_idxs_list = list(selector.get_support(indices=True))
column_names = [num_feats_list[i] for i in selected_feats_idxs_list]
column_names

c:\Users\vchar\anaconda3\envs\ml_projects\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['Ceiling Height',
 'Floor',
 'New Construction',
 'Construction Type',
 'Number of Bathrooms',
 'Floors in the Building',
 'air conditioner',
 'dishwasher',
 'washing machine',
 'fridge',
 'drying machine',
 'stove',
 'concierge',
 'intercom entry',
 'is_capital']

In [75]:
class FSelector():

    def __init__(self, X, y, num_feats, ordinal_feats, nominal_feats, model):

        self.X = X
        self.y = y
        self.num_feats = num_feats
        self.ordinal_feats = ordinal_feats
        self.nominal_feats = nominal_feats
        self.model = model

    def calculate_vif(self, X):
    
        vif = pd.DataFrame()
        vif["features"] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

        return vif

    def select_feats_via_vif(self):

        num_features = self.num_feats.copy()

        vif_df = self.calculate_vif(self.X[num_features])

        while vif_df[vif_df['VIF']>=10].shape[0] != 0:
            vif_df.sort_values('VIF', ascending=False, inplace=True)
            vif_df.reset_index(drop=True, inplace=True)
            # print(vif_df)
            elimination_candidate = vif_df.iloc[0]['features']
            # print(elimination_candidate)
            num_features = [i for i in num_features if i!=elimination_candidate]
            new_X = self.X[num_features]
            vif_df = self.calculate_vif(new_X)

        return list(vif_df['features'].values)
    
    def get_spearmanr(self):
        return [stats.spearmanr(self.X.values[:, f], self.y.values).correlation for f in range(self.X.shape[1])]
    
    def get_kendalltau(self):
        return [stats.kendalltau(self.X.values[:, f], self.y.values).correlation for f in range(self.X.shape[1])]
    
    def get_pointbiserialr(self):
        return [stats.pointbiserialr(self.X.values[:, f], self.y.values).correlation for f in range(self.X.shape[1])]
    
    def get_boruto_feats(self, model):
        feat_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=1)
        feat_selector.fit(np.array(self.X), np.array(self.y))
        return feat_selector.support_
    
    def get_kbest(self, X, feats_list, metric, k=15):
        selector = SelectKBest(metric, k=k)
        selector.fit_transform(X[feats_list], self.y)
        selected_feats_idxs_list = list(selector.get_support(indices=True))
        column_names = [num_feats_list[i] for i in selected_feats_idxs_list]
        return column_names
    
    def get_votes(self):

        if self.num_feats is not None:

            self.vif_feats = self.select_feats_via_vif()

            self.pearson_feats = self.get_kbest(X=self.X, feats_list=self.num_feats, metric=r_regression, k=15)
            # self.num_spearmanr_feats = self.get_kbest(X=self.X, feats_list=self.num_feats, metric=stats.spearmanr, k=15)
            # self.num_kendalltau_feats = self.get_kbest(X=self.X, feats_list=self.num_feats, metric=stats.kendalltau, k=15)

            # self.num_spearmanr_feats = SelectKBest(self.get_spearmanr, k=15).fit_transform(self.X[self.num_feats], self.y)
            # self.num_kendalltau_feats = SelectKBest(self.get_kendalltau, k=15).fit_transform(self.X[self.num_feats], self.y)

            self.selected_num_feats = []
            self.selected_num_feats.extend(self.pearson_feats)
            # self.selected_num_feats.extend(self.num_spearmanr_feats)
            # self.selected_num_feats.extend(self.num_kendalltau_feats)
            # self.selected_num_feats = list(set(self.selected_num_feats))

        if self.ordinal_feats is not None:

            # self.ordinal_spearmanr_feats = self.get_kbest(X=self.X, feats_list=self.ordinal_feats, metric=stats.spearmanr, k=15)
            # self.ordinal_kendalltau_feats = self.get_kbest(X=self.X, feats_list=self.ordinal_feats, metric=stats.kendalltau, k=15)

            # self.ordinal_spearmanr_feats = SelectKBest(self.get_spearmanr, k=15).fit_transform(self.X[self.ordinal_feats], self.y)
            # self.ordinal_kendalltau_feats = SelectKBest(self.get_kendalltau, k=15).fit_transform(self.X[self.ordinal_feats], self.y)

            self.selected_ordinal_feats = []
            self.selected_ordinal_feats.extend(self.ordinal_spearmanr_feats)
            self.selected_ordinal_feats.extend(self.ordinal_kendalltau_feats)
            # self.selected_ordinal_feats = list(set(self.selected_ordinal_feats))

        if self.nominal_feats is not None:

            # self.f_feats = self.get_kbest(X=self.X, feats_list=self.nominal_feats, metric=f_classif, k=15)
            # self.mi_feats = self.get_kbest(X=self.X, feats_list=self.nominal_feats, metric=mutual_info_regression, k=15)

            # # self.f_feats = f_classif(self.X[self.nominal_feats], self.y)[0]
            # self.f_feats = SelectKBest(f_classif, k=15).fit_transform(self.X[self.nominal_feats], self.y).columns
            
            # # self.mi_feats = mutual_info_regression(self.X[self.nominal_feats], self.y)
            # self.mi_feats = SelectKBest(mutual_info_regression, k=15).fit_transform(self.X[self.nominal_feats], self.y).columns

            self.selected_nominal_feats = []
            self.selected_nominal_feats.extend(self.f_feats)
            self.selected_nominal_feats.extend(self.mi_feats)
            # self.selected_nominal_feats = list(set(self.selected_nominal_feats))

        if self.model is not None:
            # np.int = np.int32
            # np.float = np.float64
            # np.bool = np.bool_
            self.boruto_feats =  self.get_boruto_feats(self.model)

        self.selected_num_feats.extend(self.boruto_feats)
        num_feats_dict = dict(Counter(self.selected_num_feats))
        self.selected_num_feats = [i for i in num_feats_dict if num_feats_dict[i] >= 2]

        # self.selected_ordinal_feats.extend(self.boruto_feats)
        # ordinal_feats_dict = dict(Counter(self.selected_ordinal_feats))
        # self.selected_ordinal_feats = [i for i in ordinal_feats_dict if ordinal_feats_dict[i] >= 2]

        # self.selected_nominal_feats.extend(self.boruto_feats)
        # nominal_feats_dict = dict(Counter(self.selected_nominal_feats))
        # self.selected_nominal_feats = [i for i in nominal_feats_dict if nominal_feats_dict[i] >= 2]

        self.selected_feats = []
        self.selected_feats.extend(self.selected_num_feats)
        # self.selected_feats.extend(self.selected_ordinal_feats)
        # self.selected_feats.extend(self.selected_nominal_feats)
        self.selected_feats.extend(self.boruto_feats)

        return self.selected_feats

In [ ]:
fselector.get_boruto_feats(model)

In [76]:
# fselector = FSelector(
#     X=X, 
#     y=y, 
#     num_feats=num_feats_list, 
#     ordinal_feats=ordinal_feats_list, 
#     nominal_feats=nominal_feats_list, 
#     model=model
# )

# fselector.select_feats_via_vif()

In [78]:
# models_list = [RandomForestRegressor(), XGBRegressor(), LGBMRegressor()]
# model_names_list = ['RandomForestRegressor', 'XGBRegressor', 'LGBMRegressor']

# for i in range(len(models_list)):

#     model = models_list[i]
    
#     fselector = FSelector(
#         X=X, 
#         y=y, 
#         num_feats=num_feats_list, 
#         ordinal_feats=None, 
#         nominal_feats=None, 
#         model=model
#     )

#     selected_feats_list = fselector.get_votes()

#     print(f"{model_names_list}: {selected_feats_list}")

In [54]:
rf = RandomForestRegressor()
rf_rfe = RFE(rf, n_features_to_select=15)
rf_rfe.fit(X, y)
# X_selection = rf_rfe.transform(X)

In [65]:
rf_rfe_feats = X.iloc[:, list(rf_rfe.support_)].columns
rf_rfe_feats

Index(['Renovation', 'Furniture', 'Construction Type', 'Ceiling Height',
       'Floor', 'Floor Area', 'Floors in the Building', 'is_capital',
       'latitude', 'longitude', 'city_attr_3', 'city_attr_4', 'city_attr_8',
       'mall_2', 'closest_mall_distance'],
      dtype='object')

In [55]:
xgbr = XGBRegressor()
xgb_rfe = RFE(xgbr, n_features_to_select=15)
xgb_rfe.fit(X, y)
# X_selection = xgb_rfe.transform(X)

In [66]:
xgb_rfe_feats = X.iloc[:, list(xgb_rfe.support_)].columns
xgb_rfe_feats

Index(['Renovation', 'Furniture', 'New Construction', 'Construction Type',
       'Number of Bathrooms', 'Elevator', 'Ceiling Height', 'Floor Area',
       'Floors in the Building', 'air conditioner', 'view of Ararat',
       'concierge', 'intercom entry', 'playground', 'is_capital'],
      dtype='object')

In [56]:
lgbmr = LGBMRegressor()
lgbmr_rfe = RFE(lgbmr, n_features_to_select=15)
lgbmr_rfe.fit(X, y)
# X_selection = lgbmr_rfe.transform(X)

In [67]:
lgbmr_rfe_feats = X.iloc[:, list(lgbmr_rfe.support_)].columns
lgbmr_rfe_feats

Index(['Number of Rooms', 'Renovation', 'Furniture', 'Construction Type',
       'Balcony', 'Ceiling Height', 'Floor', 'Floor Area',
       'Floors in the Building', 'latitude', 'longitude',
       'closest_metro_distance', 'mall_1', 'mall_2', 'closest_mall_distance'],
      dtype='object')

In [84]:
selector = BorutaShap(importance_measure='shap', classification=False)
selector.fit(X=X, y=y, n_trials=100, sample=False, verbose=True)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
features_to_remove = selector.features_to_remove
boruta_shap_selected_feats = [col for col in feats_name_list if col not in features_to_remove]
boruta_shap_selected_feats

## Fold creation

In [ ]:
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# cv_split_list = []

# for train_idxs, valid_idxs in kfold.split(X):
#     cv_split_list.append((train_idxs, valid_idxs))

kf = KFold(n_splits=10, shuffle=True, random_state=13)
kf_cv_list = [idxs for idxs in kf.split(X)]

In [ ]:
rf_rfecv = RFECV(
    RandomForestRegressor(),
    cv=kf_cv_list
)
rf_rfecv.fit(X, y)

rf_rfecv_feats = X.iloc[:, list(rf_rfecv.support_)].columns
rf_rfecv_feats

In [ ]:
xgb_rfecv = RFECV(
    XGBRegressor(),
    cv=kf_cv_list
)
xgb_rfecv.fit(X, y)

xgb_rfecv_feats = X.iloc[:, list(xgb_rfecv.support_)].columns
xgb_rfecv_feats

In [ ]:
lgbm_rfecv = RFECV(
    LGBMRegressor(),
    cv=kf_cv_list
)
lgbm_rfecv.fit(X, y)

lgbm_rfecv_feats = X.iloc[:, list(lgbm_rfecv.support_)].columns
lgbm_rfecv_feats

# Model construction

## Random Forest

In [ ]:
n_estimators_list = [25, 50, 100, 120, 300, 500]#, 800, 1200]

max_depth_list = [5, 8, 15, 25, 30, None]

min_samples_split_list = [2, 5, 10, 15, 100]

min_samples_leaf_list = [2, 5, 10]

max_features_list = ['log2', 'sqrt', None]

params_dict={
    'n_estimators': n_estimators_list,
    'max_depth': max_depth_list,
    'min_samples_split': min_samples_split_list,
    'min_samples_leaf': min_samples_leaf_list,
    'max_features': max_features_list
}

# rf_gscv = GridSearchCV(
#     RandomForestRegressor(),
#     param_grid=params_dict,
#     scoring='neg_mean_squared_error',
#     cv=kf_cv_list,
#     n_jobs=-1,
#     verbose=4
# )

rf_rscv = RandomizedSearchCV(
    RandomForestRegressor(),
    param_distributions=params_dict,
    scoring='neg_mean_squared_error',
    cv=kf_cv_list,
    n_jobs=-1,
    verbose=4,
    n_iter=30,
    random_state=42
)

rf_rscv.fit(X[rf_rfecv_feats], y)

print(rf_rscv.best_params_)
print(-rf_rscv.best_score_)

## XGBoost

In [ ]:
eta_list = [0.01, 0.015, 0.025, 0.05, 0.1, 0.3]

gamma_list = [0, 0.05, 0.07, 0.09, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]

max_depth_list = [3, 5, 6, 7, 9, 12, 15, 17, 25]

min_child_weight_list = [1, 3, 5, 7]

subsample_list = [0.6, 0.7, 0.8, 0.9, 1.0]

colsample_bytree_list = [0.6, 0.7, 0.8, 0.9, 1.0]

lambda_list = [0.01, 0.03, 0.1, 1.0]

alpha_list = [0, 0.1, 0.5, 1.0]


params_dict={
    'eta': eta_list,
    'gamma': gamma_list,
    'max_depth': max_depth_list,
    'min_child_weight': min_child_weight_list,
    'subsample': subsample_list,
    'colsample_bytree': colsample_bytree_list,
    'lambda': lambda_list,
    'alpha': alpha_list
}


xgb_rscv = RandomizedSearchCV(
    XGBRegressor(),
    param_distributions=params_dict,
    scoring='neg_mean_squared_error',
    cv=kf_cv_list,
    n_jobs=-1,
    verbose=4,
    n_iter=30,
    random_state=42
)

xgb_rscv.fit(X[xgb_rfecv_feats], y)

print(xgb_rscv.best_params_)
print(-xgb_rscv.best_score_)

In [ ]:
np.random.seed(42)

params_dict ={
    'num_leaves': [i for i in range(1, 100)],
    #'min_child_samples': np.random.randint(100, 500),
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
    #'subsample': sp_uniform(loc=0.2, scale=0.8),
    #'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
    'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
    'max_depth': [-1, 1, 2, 3, 4, 5, 6, 7],
    'n_estimators': [i for i in range(5, 1000, 10)],
    'learning_rate': [0.0001,0.0005,0.001,0.005,0.01,0.05,0.1],
    'feature_fraction': [0.1, 0.5], #set=, subsample= will be ignored
    'bagging_fraction': [0.1, 0.5], #set=,subsample= will be ignored
    'min_data_in_leaf': [i for i in range(10, 500, 10)], #set=, min_child_samples=
}


# params_dict = {
#     'bagging_fraction': (0.5, 0.8),
#     'bagging_frequency': (5, 8),
#     'feature_fraction': (0.5, 0.8),
#     'max_depth': (10, 13),
#     'min_data_in_leaf': (90, 120),
#     'num_leaves': (1200, 1550)
# }


lgbm_rscv = RandomizedSearchCV(
    LGBMRegressor(),
    param_distributions=params_dict,
    scoring='neg_mean_squared_error',
    cv=kf_cv_list,
    n_jobs=-1,
    verbose=4,
    n_iter=30,
    random_state=42
)

lgbm_rscv.fit(X[lgbm_rfecv_feats], y)

print(lgbm_rscv.best_params_)
print(-lgbm_rscv.best_score_)

In [ ]:
# remove numeric features with low variance
# use vif to eliminate correlated features
# use mutual_info for preliminary feature elimination
# RandomForest: use scores to eliminate features
# XGBoost: use scores to eliminate features
# LightGBM: use scores to eliminate features

# create one hot features
# Lasso: eliminate features with 0 coefficients
# SVM: 

In [ ]:
# num_pipeline = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(strategy='median')),
#         ('scaler', StandardScaler())
#     ]
# )

# cat_pipeline = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('ordinalencoder', OrdinalEncoder(categories=[list different categories here])),
#         ('scaler', StandardScaler())
#     ]
# )

# preprocessor = ColumnTransformer(
#     ('num_pipeline', num_pipeline, numerical_cols),
#     ('cat_pipeline', cat_pipeline, cat_cols)
# )

# preprocessor.fit_transform(), preprocessor.get_feature_names_out()